# Optimizing realsense parameters for data acquisition

The Intel Realsense D415 comes supplied with many parameters that affect the quality of the depth acquisition. This can be a huge pain to tune by hand.

The goal of this notebook is to use python bindings to the realsense library and optimize parameters to improve the quality of our mouse recordings.

In [6]:
# first make sure the realsense python bindings are importable by python
# download libreasense sdk using the following instructions (For Linux):

# 1. open terminal
# 2. $ git clone https://github.com/IntelRealSense/librealsense
# 3. $ cd librealsense
# 4. $ mkdir build
# 5. $ cd build

# in order to get import pyrealsense2 to work you need to cmake with bindings that specify the full path of the python executable
# in order to find the full path, open python3 in the terminal:

# 6. $ python3  #this opens the python environment
# 7. >>>import sys
# 8. >>>print(sys.executable)
#       /usr/bin/python
# 9. >>>quit()

# if you are working in ipython notebook, you can do the above commands right in the notebook

In [15]:
import sys
print(sys.executable)

/Users/ranyoha/anaconda3/bin/python


In [16]:
# now, when you use cmake (remember you're still in the build directory), copy the full path after -DBUILD_PYTHON_EXECUTABLE=[full path] as follows:

# 10. $ cmake ../ -DBUILD_PYTHON_BINDINGS=TRUE -DBUILD_PYTHON_EXECUTABLE=/home/rockwell/miniconda3/bin/python
# 11. $ make -j4
# 12. $ sudo make install #Optional if you want the library to be installed in your system

In [14]:
sys.path.append('/usr/local/lib')

In [13]:
import pyrealsense2 as rs

ModuleNotFoundError: No module named 'pyrealsense2'

In [11]:
import numpy as np
import cv2
import json
import time

In [7]:
# Enabling Advanced Mode

DS5_product_ids = ["0AD1", "0AD2", "0AD3", "0AD4", "0AD5", "0AF6", "0AFE", "0AFF", "0B00", "0B01", "0B03", "0B07"]

def find_device_that_supports_advanced_mode() :
    ctx = rs.context()
    ds5_dev = rs.device()
    devices = ctx.query_devices();
    for dev in devices:
        if dev.supports(rs.camera_info.product_id) and str(dev.get_info(rs.camera_info.product_id)) in DS5_product_ids:
            if dev.supports(rs.camera_info.name):
                print("Found device that supports advanced mode:", dev.get_info(rs.camera_info.name))
            return dev
    raise Exception("No device that supports advanced mode was found")

try:
    dev = find_device_that_supports_advanced_mode()
    advnc_mode = rs.rs400_advanced_mode(dev)
    print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    # Loop until we successfully enable advanced mode
    while not advnc_mode.is_enabled():
        print("Trying to enable advanced mode...")
        advnc_mode.toggle_advanced_mode(True)
        # At this point the device will disconnect and re-connect.
        print("Sleeping for 5 seconds...")
        time.sleep(5)
        # The 'dev' object will become invalid and we need to initialize it again
        dev = find_device_that_supports_advanced_mode()
        advnc_mode = rs.rs400_advanced_mode(dev)
        print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    advnc_mode.load_json(J) #this line is actually not necessary

except Exception as e:
    print(e)
pass

Found device that supports advanced mode: Intel RealSense D415
Advanced mode is enabled
name 'J' is not defined


In [4]:
# Loading presaved parameters 
with open("d415paramset_1.json") as json_data: #Json file to be loaded
    J = json.load(json_data)
    J=str(J).replace("'", '\"') #IF YOU DON't DO THIS IT WILL NOT WORK

In [5]:
print(J)

{"param-disableraucolor": "0", "param-disablesadcolor": "0", "param-disablesadnormalize": "0", "param-disablesloleftcolor": "0", "param-disableslorightcolor": "0", "param-lambdaad": "800", "param-lambdacensus": "26", "param-leftrightthreshold": "24", "param-maxscorethreshb": "2047", "param-medianthreshold": "500", "param-minscorethresha": "1", "param-neighborthresh": "7", "param-robbinsmonrodecrement": "10", "param-robbinsmonroincrement": "10", "param-secondpeakdelta": "325", "param-texturecountthresh": "0", "param-texturedifferencethresh": "0", "param-raumine": "1", "param-rauminn": "1", "param-rauminnssum": "3", "param-raumins": "1", "param-rauminw": "1", "param-rauminwesum": "3", "param-regionshrinku": "3", "param-regionshrinkv": "1", "param-regioncolorthresholdr": "0.05", "param-regioncolorthresholdg": "0.05", "param-regioncolorthresholdb": "0.05", "param-rsmdiffthreshold": "4", "param-rsmrauslodiffthreshold": "1", "param-rsmremovethreshold": "0.375", "param-usersm": "1", "param-sc

In [9]:
#Experimental Loop

frame_width=640
frame_height=480
fps=30
number_of_frames_to_record=180
file = open('pythondepthdata.bin', 'a+b') #open a file for apending binary data
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, frame_width, frame_height, rs.format.z16, fps)
config.enable_stream(rs.stream.color, frame_width, frame_height, rs.format.bgr8, fps)

# Start streaming
cfg = pipeline.start(config)
dev = cfg.get_device()
depth_sensor = dev.first_depth_sensor()
#depth_sensor.set_option(rs.option.visual_preset, 2)


iteration = 0;
preset = 0;
preset_name = '';
tick=0; #keeping track of the number of loops the experiment has gone through

start_of_loop = time.time()
with open('pythondepthdata.bin','ab') as binarydata: #While the file called "pythonedpthdata.bin" is open for appending binary, 
    try:
        while tick<number_of_frames_to_record: #aiming for 200 frames
            tick=tick+1 
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
        
            #iteration = iteration + 1
            #if iteration > 100:
               #preset = preset + 1
               #iteration = 0
               #range = depth_sensor.get_option_range(rs.option.visual_preset)
               #preset = preset % range.max
               #depth_sensor.set_option(rs.option.visual_preset, preset)
               #preset_name = depth_sensor.get_option_value_description(rs.option.visual_preset, preset)
        
            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())
            depth_image.tofile(binarydata) #save data as binarydata (specified by the with open as line)
        
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, None, 0.5, 0), cv2.COLORMAP_JET)

            # Stack both images horizontally
            images = np.hstack((color_image, depth_colormap))
        
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(images, preset_name,(60,80), font, 4,(255,255,255),2,cv2.LINE_AA) 

            # Show images
            cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('RealSense', images)
            cv2.waitKey(1)

    finally:

        # Stop streaming
        pipeline.stop()
        end_of_loop=time.time() #end timer
        print("Number of Experimental Loops = ", tick)
        print()
        print("Time Elapsed = ", end_of_loop - start_of_loop)
        serialized_string_2 = advnc_mode.serialize_json()
        as_json_object = json.loads(serialized_string_2)
        json_used_in_experiment = str(as_json_object).replace("'", '\"')
        print()
        print("Parameters Used = ", json_used_in_experiment)
       
      

Number of Experimental Loops =  180

Time Elapsed =  6.243042469024658

Parameters Used =  {"aux-param-autoexposure-setpoint": "400", "aux-param-colorcorrection1": "0.461914", "aux-param-colorcorrection10": "-0.553711", "aux-param-colorcorrection11": "-0.553711", "aux-param-colorcorrection12": "0.0458984", "aux-param-colorcorrection2": "0.540039", "aux-param-colorcorrection3": "0.540039", "aux-param-colorcorrection4": "0.208008", "aux-param-colorcorrection5": "-0.332031", "aux-param-colorcorrection6": "-0.212891", "aux-param-colorcorrection7": "-0.212891", "aux-param-colorcorrection8": "0.68457", "aux-param-colorcorrection9": "0.930664", "aux-param-depthclampmax": "65536", "aux-param-depthclampmin": "624", "aux-param-disparityshift": "53", "controls-autoexposure-auto": "True", "controls-autoexposure-manual": "33000", "controls-color-autoexposure-auto": "True", "controls-color-autoexposure-manual": "166", "controls-color-backlight-compensation": "0", "controls-color-brightness": "0", "c

In [10]:
!wc -c pythondepthdata.bin #count number of bytes in the file we just created. each pixel is a uint16 so divide bytes/framedimensions/2

110592000 pythondepthdata.bin


In [11]:
110592000/640/480/2 #should equal number of frames to record AND tick

200.0

In [12]:
#NOICE